# Goal 3: Investigate overall correlation between quantitative and qualitative metrics with a classification based approach

**3.1** 
- Which metrics are better to classify correctly the experiment? Use external labels [Good, Mid, Bad]
- Train a classifier to predict the goodness of the experiment based on assigned labels using both QM and HM features.

**3.2**
 - Use HM survey data as labels: An overall label [0, 1, 2] assigned by humans for each experiment can be found with clustering or with weighted average
 - Train a classifier with x = QM and y = HM survey metrics

**Ablation (both cases)**: try all the combination of QM to get the best classification accuracy

**Experiments Scenarios**:
- "Passing"
- "Overtaking"
- "Crossing 1"
- "Crossing 2"
- "Advanced 1"
- "Advanced 2"
- "Advanced 3"
- "Advanced 4"

**Labels**:
- "Good"
- "Mid"
- "Bad"

**QM Metrics**
- [0] Time to Goal
- [1] Path length
- [2] Cumulative heading changes
- [3] Avg robot linear speed

- [4] Social Work 
- [5] Social Work (per second)
- [6] Average minimum distance to closest person
- [7] Proxemics: intimate space occupancy
- [8] Proxemics: personal space occupancy
- [9] Proxemics: social space occupancy
- [10] Proxemics: public space occupancy

**HM Metrics**
- [0] Unobtrusiveness
- [1] Friendliness
- [2] Smoothness
- [3] Avoidance Foresight

In [1]:
import yaml
import numpy as np
import pandas as pd
import os
from os.path import expanduser

In [2]:
home = expanduser("~")
# Load config params for experiments
config = yaml.safe_load(open('params.yaml'))['social_metrics_match']

lab_data_path = home + config['data']['repo_dir'] + config['data']['lab_data_path']
survey_data_path = home + config['data']['repo_dir'] + config['data']['survey_data_path']
results_dir = home + config['data']['results_path']
print("lab data path: ", lab_data_path)
print("survey data path: ", survey_data_path)
print("results dir path: ", results_dir)

lab data path:  /root/Social-Nav-Metrics-Matching/social_metrics_match/data_folder/validation_of_metrics_quantitative_and_lab_qualitative.ods
survey data path:  /root/Social-Nav-Metrics-Matching/social_metrics_match/data_folder/qualitative_metrics_survey.xlsx
results dir path:  /root/social_metrics_results


# Extract LAB data arrays

In [3]:
from utils.data_organization import organize_dict_lab_data, get_all_lab_data_arr, np_extract_exp_lab, np_single_lab_run
from utils.data_organization  import organize_dict_survey, weighted_avg_survey_data, get_robotics_knowledge, datacube_qual_survey_data

In [4]:
dict_lab_data = organize_dict_lab_data(lab_data_path)

# Extract the np arrays of a specific experiments identified by its keys
passing_good_QM_array, passing_good_HM_array = np_single_lab_run(dict_lab_data, experiment='Passing', label='Good')
print(f"Passing single run QM shape:{passing_good_QM_array.shape}, passing single run HM shape: {passing_good_HM_array.shape}")
print(f"Passing good QM: {passing_good_QM_array},\nPassing good HM: {passing_good_HM_array}") 

# Extract the np arrays of a lab scenario (all the 3 runs with different labels), dividing QM and HM
passing_QM_array, passing_HM_array = np_extract_exp_lab(dict_lab_data, experiment='Advanced 4', order=True, normalize=True, normalization="rescale")
print(f"passing QM shape:{passing_QM_array.shape}, passing HM shape: {passing_HM_array.shape}")
# print(f"passing QM: {passing_QM_array},\npassing HM: {passing_HM_array}")

# Starting from the complete dataframe with lab data, Extract the np arrays of all lab scenarios dividing QM and HM
all_lab_QM_array, all_lab_HM_array = get_all_lab_data_arr(dict_lab_data, order=True, normalize=True, normalization="rescale")
print(f"All lab QM array: {all_lab_QM_array.shape}, All lab HM array: {all_lab_HM_array.shape}")
# print(f"All lab QM array: {all_lab_QM_array}, All lab HM array: {all_lab_HM_array}")

Passing single run QM shape:(11,), passing single run HM shape: (4,)
Passing good QM: [1.02326076e+01 4.55598068e+00 4.04913597e+00 1.99936767e-01
 1.99432749e+03 1.81716719e+02 2.37741413e+00 9.28961749e+00
 1.23341140e+01 7.28337237e+01 5.54254489e+00],
Passing good HM: [0.8 0.8 0.8 1. ]
passing QM shape:(11, 3), passing HM shape: (4, 3)
All lab QM array: (24, 11), All lab HM array: (24, 4)


**SURVEY DATA**

In [5]:
dict_survey_data = organize_dict_survey(survey_data_path)
robot_knowledge_array = get_robotics_knowledge(survey_data_path)

# To extract np arrays of all the survey data
survey_datacube = datacube_qual_survey_data(dict_survey_data, normalize=True)

# To directly extract the average and std: If Weighted average set w_avg=True (use robotics background knowledge as weights)
weighted_survey_array_avg, weighted_survey_array_std = weighted_avg_survey_data(dict_survey_data, robot_knowledge_array, w_avg=True, normalize=False)
print(f"survey weighted avg shape: {weighted_survey_array_avg.shape},\nsurvey weighted std shape:  {weighted_survey_array_std.shape}") 

survey weighted avg shape: (24, 4),
survey weighted std shape:  (24, 4)


In [6]:
from sklearn.metrics import accuracy_score

# Encode survey aggregated scores in labels format
survey_score = np.rint(np.nanmean(weighted_survey_array_avg, axis=1)).astype(int)
survey_score_coded = survey_score.copy()
survey_score_coded[survey_score_coded < 3] = 0
survey_score_coded[survey_score_coded == 3] = 1
survey_score_coded[survey_score_coded > 3] = 2
print(survey_score_coded)

# Manual labels assigned by us as [Good, Intermediate, Bad]
manual_labels=["Bad","Intermediate","Good","Good","Intermediate","Bad","Intermediate","Bad","Good","Intermediate","Good","Bad",
                   "Bad","Intermediate","Good","Good","Intermediate","Bad","Intermediate","Bad","Good","Intermediate","Good","Bad"]
manual_encoded = pd.Categorical(manual_labels, categories=["Bad", "Intermediate", "Good"]).codes
print(manual_encoded)

accuracy_score(survey_score_coded, manual_encoded)

[0 1 2 1 1 2 2 0 2 2 1 0 1 2 2 1 1 0 2 0 2 0 0 0]
[0 1 2 2 1 0 1 0 2 1 2 0 0 1 2 2 1 0 1 0 2 1 2 0]


0.5416666666666666

# Classifier study

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
# Define X, y dataset
# X = QM metrics combination
# y = HM survey aggregatede scores

X = all_lab_QM_array
y = np.rint(weighted_survey_array_avg).astype(int) # manual_encoded or survey_score_coded

X_train = np.vstack([X[:15, :], X[-3:, :]])
y_train = np.vstack([y[:15, :], y[-3:, :]])
print(X_train.shape, y_train.shape)

X_test = X[15:21, :]
y_test = y[15:21]
print(X_test.shape, y_test.shape)

(18, 11) (18, 4)
(6, 11) (6, 4)


In [9]:
# Define Decision Tree Classifier
print("Random Forest Classifier")

rf_clf = RandomForestClassifier(max_depth=2, n_estimators = 25, criterion = 'entropy', random_state = 42)
rf_clf.fit(X_train, y_train)

# Predicting the Test set results
y_pred = rf_clf.predict(X_test)
print("Y pred: ", y_pred)
print("Y true: ", y_test)

for i in range(y_test.shape[1]):
    print(accuracy_score(y_test[:, i], y_pred[:, i]))

Random Forest Classifier
Y pred:  [[4 4 3 3]
 [4 3 3 3]
 [4 4 3 3]
 [4 4 3 4]
 [4 3 3 3]
 [4 3 3 4]]
Y true:  [[3 3 3 3]
 [2 3 3 3]
 [2 2 2 2]
 [4 4 4 4]
 [2 2 2 2]
 [4 4 4 4]]
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.6666666666666666


In [10]:
from utils.cluster_evaluation import get_combination_QM_clf, get_combination_QM_multiout_clf

# res = get_combination_QM_clf(X ,y)
res_multi = get_combination_QM_multiout_clf(X ,y)

QM metrics columns: (0, 6) mean_acc: 0.6666666666666667 ACC: [0.5, 0.8333333333333334, 0.5, 0.8333333333333334]
QM metrics columns: (0, 1, 6) mean_acc: 0.625 ACC: [0.6666666666666666, 0.6666666666666666, 0.3333333333333333, 0.8333333333333334]
QM metrics columns: (0, 2, 6, 9, 10) mean_acc: 0.625 ACC: [0.5, 0.6666666666666666, 0.5, 0.8333333333333334]
QM metrics columns: (1, 2, 3, 6, 7, 9) mean_acc: 0.625 ACC: [0.6666666666666666, 0.6666666666666666, 0.3333333333333333, 0.8333333333333334]
QM metrics columns: (0, 2, 3, 4, 6) mean_acc: 0.5833333333333334 ACC: [0.5, 0.8333333333333334, 0.3333333333333333, 0.6666666666666666]
QM metrics columns: (0, 3, 6) mean_acc: 0.5833333333333333 ACC: [0.5, 0.6666666666666666, 0.5, 0.6666666666666666]
QM metrics columns: (0, 1, 2, 6) mean_acc: 0.5833333333333333 ACC: [0.5, 0.6666666666666666, 0.3333333333333333, 0.8333333333333334]
QM metrics columns: (0, 1, 5, 6) mean_acc: 0.5833333333333333 ACC: [0.5, 0.6666666666666666, 0.3333333333333333, 0.8333333

In [11]:
# Define Decision Tree Classifier
# print("Decision Tree Classifier")

# dt_clf = DecisionTreeClassifier(max_depth=2, criterion = 'entropy', random_state = 42)
# dt_clf.fit(X_train, y_train)

# # Predicting the Test set results
# y_pred = dt_clf.predict(X_test)
# print("Y pred: ", y_pred)
# print("Y true: ", y_test)

# accuracy_score(y_test, y_pred)

In [12]:
# Define SVM Classifier
# print("SVM Classifier")
# svm_clf = SVC(decision_function_shape='ovr', kernel='rbf')
# svm_clf.fit(X_train, y_train)

# # Predicting the Test set results
# y_pred = svm_clf.predict(X_test)
# print("Y pred: ", y_pred)
# print("Y true: ", y_test)

# accuracy_score(y_test, y_pred)

GridSearch for hyperparameters tuning

In [13]:
# n_estimators = [10, 25]
# criterion = ['gini', 'entropy']
# bootstrap = [True, False]

# param_grid = {
#     "n_estimators": n_estimators,
#     "bootstrap": bootstrap,
#     "criterion":criterion,
# }

# rf = RandomForestClassifier(random_state=42)

# rf_model = GridSearchCV(estimator=rf, param_grid=param_grid, cv=2, verbose=1, n_jobs=-1)
# rf_model.fit(X_train, y_train)

# print("Using hyperparameters --> \n", rf_model.best_params_)